# Odczyty uliniowione

W tej części zajmiemy się analiża uliniowionych odczytów (BAM) poprzez platformę Spark. Tym razem zamiast korzystać z API DataFrame posłużymy się językiem SQL.

In [2]:
! gsutil ls gs://edugen-lab-$USER/bam

gs://edugen-lab-agaszmurlo/bam/
gs://edugen-lab-agaszmurlo/bam/.DS_Store
gs://edugen-lab-agaszmurlo/bam/father.bai
gs://edugen-lab-agaszmurlo/bam/father.bam
gs://edugen-lab-agaszmurlo/bam/mother.bai
gs://edugen-lab-agaszmurlo/bam/mother.bam
gs://edugen-lab-agaszmurlo/bam/mother.bam.bai
gs://edugen-lab-agaszmurlo/bam/mother.bam.sbi
gs://edugen-lab-agaszmurlo/bam/mother10.bam
gs://edugen-lab-agaszmurlo/bam/mother10.bam.bai
gs://edugen-lab-agaszmurlo/bam/mother10.bam.sbi
gs://edugen-lab-agaszmurlo/bam/mother2.bam
gs://edugen-lab-agaszmurlo/bam/mother2.bam.bai
gs://edugen-lab-agaszmurlo/bam/mother2.bam.sbi
gs://edugen-lab-agaszmurlo/bam/mother3.bam
gs://edugen-lab-agaszmurlo/bam/mother3.bam.bai
gs://edugen-lab-agaszmurlo/bam/mother3.bam.sbi
gs://edugen-lab-agaszmurlo/bam/mother4.bam
gs://edugen-lab-agaszmurlo/bam/mother4.bam.bai
gs://edugen-lab-agaszmurlo/bam/mother4.bam.sbi
gs://edugen-lab-agaszmurlo/bam/mother6.bam
gs://edugen-lab-agaszmurlo/bam/mother6.bam.bai
gs://edugen-lab-agaszmurlo

## Przygotowanie sesji Spark
Zainicjowanie sesji Spark oraz stworzenie schematu bazy danych z której będziemy korzystać.

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

In [4]:
import os                               # moduł OS języka Python
user_name = os.environ.get('USER')      # pobieramy zmienną środowiskową USER
bucket = f"gs://edugen-lab-{user_name}" # konstruujemy sciezke dostepowa do pliku
print(bucket)

gs://edugen-lab-agaszmurlo


Będziemy korzystać z modułu Spark SQL. Możemy nasze dane traktować jako dane tabelaryczne. Zdefiniujmy tabelę.
Tym razem korzystamy z DataSource -> BAMDataSource

In [5]:
table_bam =  'alignments'

alignments_path = f"{bucket}/bam/mother10.bam"

spark.sql(f'DROP TABLE IF EXISTS {table_bam}')
spark.sql(f'CREATE TABLE IF NOT EXISTS {table_bam} \
USING org.biodatageeks.sequila.datasources.BAM.BAMDataSource \
OPTIONS(path "{alignments_path}")')


DataFrame[]

In [7]:
spark.sql(f"describe {table_bam}").show()

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|sample_id|   string|   null|
|    qname|   string|   null|
|     flag|      int|   null|
|   contig|   string|   null|
|      pos|      int|   null|
|pos_start|      int|   null|
|  pos_end|      int|   null|
|     mapq|      int|   null|
|    cigar|   string|   null|
|    rnext|   string|   null|
|    pnext|      int|   null|
|     tlen|      int|   null|
|      seq|   string|   null|
|     qual|   string|   null|
|   tag_AM|      int|   null|
|   tag_AS|      int|   null|
|   tag_BC|   string|   null|
|   tag_BQ|   string|   null|
|   tag_BZ|   string|   null|
|   tag_CB|   string|   null|
+---------+---------+-------+
only showing top 20 rows



Żeby dostać się do danych w tabeli, bedziemy korzystać z język SQL (standard ANSI). 
Struktura polecen SQL jest nastepująca: 

```sql
SELECT kolumny
FROM tabela
WHERE warunki
GROUP BY wyrazenia grupujace
HAVING warunki na grupy
ORDER BY wyrazenie sortujące
```
Zapytanie może mieć podzapytania zagnieżdzone. W zapytaniu można korzystać ze złączeń z innymi tabeli poprzez JOIN.

In [6]:
spark.sql(f"select * from {table_bam}").show()

+---------+--------------------+----+------+-------+---------+-------+----+-------------------+-----+--------+----+--------------------+--------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+---------+--------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+--------------------+------+------+------+------+
|sample_id|               qname|flag|contig|    pos|pos_start|pos_end|mapq|              cigar|rnext|   pnext|tlen|                 seq|                qual|tag_AM|tag_AS|tag_BC|tag_BQ|tag_BZ|tag_CB|tag_CC|tag_CG|tag_CM|tag_CO|tag_CP|tag_CQ|tag_CR|tag_CS|tag_CT|tag_CY|tag_E2|tag_FI|tag_FS|tag_FZ|tag_H0|tag_H1|tag_H2|tag_HI|tag_IH|tag_LB|   tag_MC|              tag_MD|tag_MI|tag_MQ|tag_NH|tag_NM|tag_OA|tag_OC|tag_OP|tag_OQ|tag_OX|tag_PG|tag_PQ|

# Wybór kolumn

In [9]:
spark.sql(f"select sample_id, contig, pos, mapq from {table_bam}").show()

+---------+------+-------+----+
|sample_id|contig|    pos|mapq|
+---------+------+-------+----+
| mother10|    20|  83235|   0|
| mother10|    20| 293889|   0|
| mother10|    20| 465705|   7|
| mother10|    20| 465710|   0|
| mother10|    20| 723766|   0|
| mother10|    20| 723766|   0|
| mother10|    20| 723766|   6|
| mother10|    20|1543878|   0|
| mother10|    20|1648876|   0|
| mother10|    20|2058293|   6|
| mother10|    20|2522293|   0|
| mother10|    20|2806380|  52|
| mother10|    20|2831118|   0|
| mother10|    20|2849265|   3|
| mother10|    20|2989376|   0|
| mother10|    20|3029659|   0|
| mother10|    20|3375591|   0|
| mother10|    20|3858310|   0|
| mother10|    20|4289927|   4|
| mother10|    20|4300760|   0|
+---------+------+-------+----+
only showing top 20 rows



In [13]:
spark.sql(f"select distinct flag from {table_bam} order by flag ASC").show()

+----+
|flag|
+----+
|  65|
|  69|
|  73|
|  81|
|  83|
|  97|
|  99|
| 113|
| 117|
| 121|
| 129|
| 133|
| 137|
| 145|
| 147|
| 161|
| 163|
| 177|
| 181|
| 185|
+----+
only showing top 20 rows



`Czym (jakim typem danych) są dane zwracane przez spark.sql ("SELECT..")?`

In [15]:
df = spark.sql(f"select distinct flag from {table_bam} order by flag ASC") 
print(type(df))
df.printSchema()

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- flag: integer (nullable = false)



### Filtrowanie wierszy 

In [11]:
spark.sql(f"select sample_id, contig, pos, mapq from {table_bam} where pos < 3858310").show()

+---------+------+-------+----+
|sample_id|contig|    pos|mapq|
+---------+------+-------+----+
| mother10|    20|  83235|   0|
| mother10|    20| 293889|   0|
| mother10|    20| 465705|   7|
| mother10|    20| 465710|   0|
| mother10|    20| 723766|   0|
| mother10|    20| 723766|   0|
| mother10|    20| 723766|   6|
| mother10|    20|1543878|   0|
| mother10|    20|1648876|   0|
| mother10|    20|2058293|   6|
| mother10|    20|2522293|   0|
| mother10|    20|2806380|  52|
| mother10|    20|2831118|   0|
| mother10|    20|2849265|   3|
| mother10|    20|2989376|   0|
| mother10|    20|3029659|   0|
| mother10|    20|3375591|   0|
+---------+------+-------+----+



In [16]:
spark.sql(f"select sample_id, contig, pos, mapq from {table_bam} where pos < 3858310 and flag NOT IN (113,117,121) and mapq > 50").show()

+---------+------+-------+----+
|sample_id|contig|    pos|mapq|
+---------+------+-------+----+
| mother10|    20|2806380|  52|
+---------+------+-------+----+



## <span style='background:yellow'> ZADANIE  </span>
Napisz zapytanie które zwróci unikalne wartości CIGAR na contigu 20 na pozycjach 1-1000000. Ile jest takich unikalnych wartości.

In [18]:
spark.sql(f"select distinct cigar from {table_bam} where pos between 1 and 1000000 and contig= '20'").count()
spark.sql(f"select count (distinct cigar) from {table_bam} where pos between 1 and 1000000 and contig= '20'").show()

7

# Grupowanie danych

Pokaż ile jest wierszy o konkretnej wartości flagi

In [23]:
spark.sql(f"select flag, count(*) as cnt from {table_bam} group by flag order by cnt desc").show()  # AS - alias kolumny

+----+-----+
|flag|  cnt|
+----+-----+
| 147|28618|
|  99|28618|
|  83|28508|
| 163|28508|
| 121|  248|
| 181|  248|
|  73|  220|
| 133|  220|
|  97|  114|
| 161|  114|
|  81|  114|
| 145|  114|
|2179|   47|
|2227|   33|
|2115|   33|
|2211|   31|
|2195|   30|
|  65|   25|
| 129|   25|
|2163|   22|
+----+-----+
only showing top 20 rows



Mozna wykorzystac kilka funkcji agregujących (na roznych kolumnach) w jednym zapytaniu. 

Pokaz ile odczytow ma dana flage. Pokaz takze ich srednia jakosc mapowania oraz minimalna pozycje na ktorej wystepuja

In [25]:
spark.sql(f"select flag, count(*) as cnt, avg(mapq) as avg_m, min(pos_start) as min from {table_bam} group by flag order by cnt, avg_m desc").show()  # AS - alias kolumny

+----+---+------------------+--------------+
|flag|cnt|             avg_m|min(pos_start)|
+----+---+------------------+--------------+
| 137|  1|              24.0|      15863674|
|2225|  1|               1.0|      25535261|
|2185|  1|               0.0|      33288211|
|  69|  1|               0.0|      15863674|
|2161|  3|               5.0|       2522293|
|2177|  3|2.3333333333333335|      32815939|
|2145|  4|              6.25|      10024268|
| 113|  5|              48.0|      10111665|
| 177|  5|              38.8|      10144245|
| 185|  5|              25.8|      10101679|
|2113|  5|              12.0|      32815939|
|2121|  5|               8.2|       5448232|
|2169|  5|               7.8|        293889|
|2129|  5|               0.0|       8986284|
| 117|  5|               0.0|      10101679|
|2209|  8|             29.25|      10111387|
|2131| 16|           16.5625|       2831118|
|2193| 18| 16.72222222222222|       7416672|
|2147| 22| 32.45454545454545|      10097473|
|2163| 22|

Funkcje mozemy zagniezdzac. Np: round(avg(kolumna))

In [26]:
# Uzycie funkcji floor na średniej
spark.sql(f"select flag, count(*) as cnt, floor(avg(mapq)) as avg_m, min(pos_start) as min from {table_bam} group by flag order by cnt, avg_m desc").show()  # AS - alias kolumny

+----+---+-----+--------------+
|flag|cnt|avg_m|min(pos_start)|
+----+---+-----+--------------+
| 137|  1|   24|      15863674|
|2225|  1|    1|      25535261|
|2185|  1|    0|      33288211|
|  69|  1|    0|      15863674|
|2161|  3|    5|       2522293|
|2177|  3|    2|      32815939|
|2145|  4|    6|      10024268|
| 113|  5|   48|      10111665|
| 177|  5|   38|      10144245|
| 185|  5|   25|      10101679|
|2113|  5|   12|      32815939|
|2121|  5|    8|       5448232|
|2169|  5|    7|        293889|
|2129|  5|    0|       8986284|
| 117|  5|    0|      10101679|
|2209|  8|   29|      10111387|
|2131| 16|   16|       2831118|
|2193| 18|   16|       7416672|
|2147| 22|   32|      10097473|
|2163| 22|    8|       8986285|
+----+---+-----+--------------+
only showing top 20 rows



## <span style='background:yellow'> ZADANIE  </span>
Wiedząc, że jako wyrażenie grupujące można wykorzystać wywołanie funkcji na kolumnie napisz zapytanie które policzy ile odczytów ma określoną długość CIGARa. Dla tych grup policz także zaokrągloną (ROUND) wartość jakości mapowania odczytów. Wynik zawęź tylko do odczytów występujących na chromosomie 20 lub 21. Posortuj malejaco liczności grup.

In [29]:
spark.sql(f"select length(cigar), count(*) as cnt, round (avg(mapq)) as avg_map \
            from {table_bam} \
            where contig IN ('20', '21') \
            group by length(cigar) \
            order by cnt desc").show()  

+-------------+-----+-------+
|length(cigar)|  cnt|avg_map|
+-------------+-----+-------+
|            4|93007|   60.0|
|            7|16142|   60.0|
|            6| 2540|   59.0|
|            9| 1866|   53.0|
|            8|  833|   58.0|
|           10|  341|   55.0|
|           11|  323|   59.0|
|           12|  216|   59.0|
|           13|  152|   59.0|
|           15|   30|   59.0|
|           14|   30|   39.0|
|           16|   27|   59.0|
|           17|   11|   58.0|
|           21|    5|   60.0|
|           18|    4|   36.0|
|           20|    3|   57.0|
|           26|    1|    3.0|
|           19|    1|    0.0|
|           22|    1|   60.0|
|           24|    1|    0.0|
+-------------+-----+-------+



# Filtrowanie grup danych

Warunki na pojedyncze wiersze nakładamy za pomocą klauzuli WHERE. Jeśli dokonujemy grupowania danych i chcemy w wynikach uzyskać jedynie informacje o grupach, które spełniają określone warunki używamy klauzuli HAVING. (HAVING moze wystapic wylacznie z GROUP BY).

In [30]:
# Zrob podsumowanie dla grup odczytów o zgodnym flag. Policz srednie mapq oraz minmalna pozycje
# Pokaz jedynie informacja o flagach dla ktorych jest ponad 1000 odczytów.
spark.sql(f"select flag, count(*) as cnt, floor(avg(mapq)) as avg_m, min(pos_start) as min \
            from {table_bam} \
            group by flag \
            having count(*) > 1000 \
            order by cnt, avg_m desc").show()

+----+-----+-----+--------+
|flag|  cnt|avg_m|     min|
+----+-----+-----+--------+
| 163|28508|   59| 9999896|
|  83|28508|   59|10000096|
| 147|28618|   59| 9998345|
|  99|28618|   59| 9998296|
+----+-----+-----+--------+



`Czy w klauzuli HAVING możemy użyc aliasu kolumny cnt?`

Warunki w HAVING też mogą być złożone (złączone AND, OR). Ale warunki w HAVING mogą się odnosić jedynie do kolumn grupujących lub funkcji agregujących

In [32]:
spark.sql(f"select flag, count(*) as cnt, floor(avg(mapq)) as avg_m, min(pos_start) as min \
            from {table_bam} \
            group by flag \
            having count(*) > 1000 and flag != 99 \
            order by cnt, avg_m desc").show()

+----+-----+-----+--------+
|flag|  cnt|avg_m|     min|
+----+-----+-----+--------+
|  83|28508|   59|10000096|
| 163|28508|   59| 9999896|
| 147|28618|   59| 9998345|
+----+-----+-----+--------+



In [34]:
# Bledna konstrukcja - W HAVING mamy warunek na kolumny niegrupujace
# spark.sql(f"select flag, count(*) as cnt, floor(avg(mapq)) as avg_m, min(pos_start) as min \
#             from {table_bam} \
#             group by flag \
#             having count(*) > 1000 and sample_id ='mother' \
#             order by cnt, avg_m desc").show()

## <span style='background:yellow'> ZADANIE  </span>
Wiedząc, że jako wyrażenie grupujące można wykorzystać wywołanie funkcji na kolumnie napisz zapytanie które policzy ile odczytów ma określoną długość CIGARa. 
Dla tych grup policz także zaokrągloną (ROUND) wartość jakości mapowania odczytów. 
Wynik zawęź tylko do odczytów występujących na chromosomie 20 lub 21. Pokaż tylko grupy mające średnia jakosc mapowania powyzej 30. Posortuj malejaco liczności grup.

In [36]:
spark.sql(f"select length(cigar), count(*) as cnt, round (avg(mapq)) as avg_map \
            from {table_bam} \
            where contig IN ('20', '21') \
            group by length(cigar) \
            having avg(mapq) > 30 \
            order by cnt desc").show()  

+-------------+-----+-------+
|length(cigar)|  cnt|avg_map|
+-------------+-----+-------+
|            4|93007|   60.0|
|            7|16142|   60.0|
|            6| 2540|   59.0|
|            9| 1866|   53.0|
|            8|  833|   58.0|
|           10|  341|   55.0|
|           11|  323|   59.0|
|           12|  216|   59.0|
|           13|  152|   59.0|
|           15|   30|   59.0|
|           14|   30|   39.0|
|           16|   27|   59.0|
|           17|   11|   58.0|
|           21|    5|   60.0|
|           18|    4|   36.0|
|           20|    3|   57.0|
|           22|    1|   60.0|
+-------------+-----+-------+



# Wykorzystanie funkcji wbudowanych

Niektóre funkcje do wykorzystania:
ROUND, FLOOR, SIGN, POW, LEAST, LOG
UPPER, LOWER, SUBSTR, 
NVL
CURRENT_DATE
MIN, MAX, SUM, STDDEV, AVG

In [37]:
spark.sql(f"select distinct sample_id, upper(sample_id), current_date() from {table_bam} ").show()

+---------+----------------+--------------+
|sample_id|upper(sample_id)|current_date()|
+---------+----------------+--------------+
| mother10|        MOTHER10|    2021-01-15|
+---------+----------------+--------------+



# Wykorzystanie funkcji rozszerzonych

Istnieją rozszerzenia Sparka do obslugi genomicznych danych. Na przykład pakiet sequila dostarcza metod do rozproszonego wyliczenia glebokosci pokrycia i pileup.

In [49]:
# zeby skorzystac z rozszerzen importujemy modul i tworzymy obiekt ss, opakowanie na sesje sparkową
from pysequila import SequilaSession
ss = SequilaSession(spark)

Coverage jest dostępny jako funkcja tabelaryczna, czyli funkcja zwracająca tabelę.

In [53]:
# konstruujemy zapytanie
cov_query = f"select * from coverage ('{table_bam}', 'mother10', 'blocks')"


In [54]:
ss.sql(cov_query).show()

+------+---------+-------+--------+
|contig|pos_start|pos_end|coverage|
+------+---------+-------+--------+
|    20|    83235|  83303|       1|
|    20|    83304| 293920|       1|
|    20|   293921| 465709|       1|
|    20|   465710| 465753|       2|
|    20|   465754| 465755|       1|
|    20|   465756| 723832|       3|
|    20|   723833| 723835|       1|
|    20|   723836|1543985|       1|
|    20|  1543986|1648918|       1|
|    20|  1648919|1648928|       1|
|    20|  1648929|2058323|       1|
|    20|  2058324|2522335|       1|
|    20|  2522336|2806409|       1|
|    20|  2806410|2831171|       1|
|    20|  2831172|2849299|       1|
|    20|  2849300|2989405|       1|
|    20|  2989406|3029670|       1|
|    20|  3029671|3029707|       1|
|    20|  3029708|3375630|       1|
|    20|  3375631|3858342|       1|
+------+---------+-------+--------+
only showing top 20 rows



## <span style='background:yellow'> ZADANIE  </span>
Pokaż tylko pozycje, które mają głębokość pokrycia większą niż 5.

In [56]:
ss.sql(f"select * from coverage ('{table_bam}', 'mother10', 'blocks') where coverage >= 5").show()

+------+---------+--------+--------+
|contig|pos_start| pos_end|coverage|
+------+---------+--------+--------+
|    20|  4553753| 4553753|       5|
|    20|  4553754| 4553795|       6|
|    20|  4553796| 4553801|       5|
|    20|  8986286| 8986330|       5|
|    20|  9999909| 9999929|       5|
|    20|  9999930| 9999944|       6|
|    20|  9999945| 9999975|       7|
|    20|  9999976| 9999978|       8|
|    20|  9999979| 9999981|       9|
|    20|  9999982| 9999986|      10|
|    20|  9999987| 9999995|      12|
|    20|  9999996| 9999996|      13|
|    20|  9999997| 9999998|      15|
|    20|  9999999|10000022|      16|
|    20| 10000023|10000026|      17|
|    20| 10000027|10000027|      18|
|    20| 10000028|10000036|      17|
|    20| 10000037|10000044|      18|
|    20| 10000045|10000053|      16|
|    20| 10000054|10000054|      15|
+------+---------+--------+--------+
only showing top 20 rows



## <span style='background:yellow'> ZADANIE  </span>
Korzystając z opcji uruchomienia funkcji coverage ktora zwraca pokrycie dla kazdej pozycji niezaleznie (bases zamiast blocks) napisz  zapytanie które zwróci ile pozycji ma daną głębokość pokrycia. Posortuj po liczności malejąco.

In [60]:
ss.sql(f"select coverage, count(*) as cnt from coverage ('{table_bam}', 'mother10', 'bases') group by coverage order by cnt desc").show()

+--------+-----+
|coverage|  cnt|
+--------+-----+
|      33|31078|
|      34|30834|
|      32|30473|
|      35|30019|
|      31|29246|
|      36|28492|
|      30|26587|
|      37|26499|
|      38|23766|
|      29|23373|
|      39|21490|
|      28|20130|
|      40|19316|
|      27|17490|
|      41|16458|
|      26|14431|
|      42|13556|
|      25|11710|
|      43|11226|
|      24| 9320|
+--------+-----+
only showing top 20 rows



In [63]:
ss.stop()

# Wykrywanie wariantów
Na zajeciach skorzystamy z wersji rozproszonej HaplotypeCaller.

In [64]:
! gatk-hpt-caller-k8s.sh \
  --conf "spark.executor.memory=1g" \
  --conf "spark.driver.memory=1g" \
  --conf "spark.executor.instances=2" \
  --conf "spark.hadoop.fs.gs.block.size=8388608" \
  -R /mnt/data/mapping/ref/ref.fasta \
  -I gs://edugen-lab-$USER/bam/mother10.bam \
  -O gs://edugen-lab-$USER/vcf/mother10.vcf

####Running GATK with users' params: --conf spark.executor.memory=1g --conf spark.driver.memory=1g --conf spark.executor.instances=2 --conf spark.hadoop.fs.gs.block.size=8388608 -R /mnt/data/mapping/ref/ref.fasta -I gs://edugen-lab-agaszmurlo/bam/mother10.bam -O gs://edugen-lab-agaszmurlo/vcf/mother10.vcf on Kubernetes
Using GATK jar /opt/gatk-4.1.9.0/gatk-package-4.1.9.0-spark.jar
Running:
    /usr/local/spark/bin/spark-submit --master k8s://https://10.3.240.1:443 --conf spark.kryoserializer.buffer.max=512m --conf spark.driver.maxResultSize=0 --conf spark.driver.userClassPathFirst=false --conf spark.io.compression.codec=lzf --conf spark.executor.memoryOverhead=600 --conf spark.driver.extraJavaOptions=-DGATK_STACKTRACE_ON_USER_EXCEPTION=true -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=false -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2  --conf spark.executor.extraJavaOptions=-DGATK_STACKTRACE_ON_USER_EXCEPTION=true -Dsamjdk.us

In [65]:
! gsutil ls gs://edugen-lab-$USER/vcf/  # sprawdzimy czy plik sie zapisal

gs://edugen-lab-agaszmurlo/vcf/
gs://edugen-lab-agaszmurlo/vcf/mother.vcf
gs://edugen-lab-agaszmurlo/vcf/mother10.vcf
gs://edugen-lab-agaszmurlo/vcf/mother_anno.vcf
gs://edugen-lab-agaszmurlo/vcf/mother_dec.vcf


In [68]:
! cat /usr/local/bin/gatk-hpt-caller-k8s.sh  # podejrzymy konfiguracje

#!/usr/bin/env bash
echo "####Running GATK with users' params: $@ on Kubernetes"
gatk HaplotypeCallerSpark \
  --spark-runner SPARK \
  --spark-master k8s://https://$KUBERNETES_SERVICE_HOST:$KUBERNETES_SERVICE_PORT \
  --conf spark.jars=/tmp/gcs-connector-${GCS_CONNECTOR_VERSION}-shaded.jar,/tmp/google-cloud-nio-${GCS_NIO_VERSION}-shaded.jar \
  --conf spark.hadoop.google.cloud.auth.service.account.enable=true \
  --conf spark.hadoop.google.cloud.auth.service.account.json.keyfile=$GOOGLE_APPLICATION_CREDENTIALS \
  --conf spark.kubernetes.driverEnv.GCS_PROJECT_ID=$PROJECT_ID \
  --conf spark.kubernetes.driverEnv.GOOGLE_APPLICATION_CREDENTIALS=$GOOGLE_APPLICATION_CREDENTIALS \
  --conf spark.executorEnv.GCS_PROJECT_ID=$PROJECT_ID \
  --conf spark.executorEnv.GOOGLE_APPLICATION_CREDENTIALS=$GOOGLE_APPLICATION_CREDENTIALS \
  --conf spark.kubernetes.driver.secrets.$SERVICE_ACCOUNT-secret=$SECRETS_MOUNT_DIR \
  --conf spark.kubernetes.executor.secrets.$SERVICE_ACCOUNT-secret=$SECRETS_MOUNT